In [351]:
import os
from dotenv import load_dotenv
from web3 import Web3
import requests
import time
from datetime import datetime

web3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/cdd85a80d6594d22b237d1c12ba9aef8'))
load_dotenv()

def getFloorPrice(collection_slug):
    response = requests.get(
        f'https://api.opensea.io/collection/{collection_slug}')
        
    return response.json()['collection']['stats']['floor_price']

def getNFTInfo(contract_address):
    checksum_contract_address = web3.toChecksumAddress('0x34d85c9CDeB23FA97cb08333b511ac86E1C4E258')
    response = requests.get(
        f'https://api.opensea.io/asset_contract/{checksum_contract_address}')
    collection = response.json()['collection']['slug']
    floor_price = getFloorPrice(collection)
    return checksum_contract_address, collection, floor_price

def login():
    url = "http://localhost:8000/login"
    payload={'username': os.getenv('AUTH_AC'),
    'password': os.getenv('AUTH_PW')}
    files=[

    ]
    headers = {}
    response = requests.request("POST", url, headers=headers, data=payload, files=files)
    if response.json()['access_token']:
        print("Login Success...")
    return response.json()

def getIndexList(access_token, token_type):
    url = "http://localhost:8000/component/all"
    payload={}
    headers = {
    'Authorization': str(token_type+' '+access_token)
    }
    response = requests.request("GET", url, headers=headers, data=payload)
    if (len(response.json())):
        print("getIndexList Success...")
    return response.json()

def checkAuth(auth):
    newAuth = login()
    print(newAuth['access_token'])
    print(auth['access_token'])
    if newAuth['access_token'] != auth['access_token']:
        auth['access_token'] = newAuth['access_token']
    else:
        print('Same Token')

def initLocalData():
    local_data = {}
    db_data = {}
    local_auth = login()
    component_raw = getIndexList(local_auth['access_token'],local_auth['token_type'])


    # collections
    local_data['collections'] = {}

    for i,component in enumerate(component_raw):
        local_data['collections'][i]={}
        local_data['collections'][i]['collection_name']=component['collection']
        local_data['collections'][i]['contract_address']=component['contract_address']
        local_data['collections'][i]['floor_price']=component['floor_price']
        local_data['collections'][i]['weighting']=component['weighting']
        local_data['collections'][i]['last_update_timestamp']=component['last_update_timestamp']
    return local_data, local_auth

def updateComponents(local_data):
    for i in local_data['collections']:
        # update
        try:
            floor_price = getFloorPrice(local_data['collections'][i]['collection_name'])
            if floor_price > 0:
                local_data['collections'][i]['floor_price'] =  floor_price 
                local_data['collections'][i]['last_update_timestamp'] = int(datetime.now().timestamp())
                print(f"Updated {local_data['collections'][i]['collection_name']} with FP {local_data['collections'][i]['floor_price']} at {local_data['collections'][i]['last_update_timestamp']}...")
        except Exception as e: 
            print(e)
        time.sleep(0.5)
        # print(index)
    return local_data
        
def feedDictConstructor(local_data):
    db_data = {}
    # blocktime
    db_data['block_number'] = web3.eth.block_number
    # timestamp
    db_data['current_timestamp'] = int(datetime.now().timestamp())
    # avg_delay
    delay = 0
    index = 0
    for i in (local_data['collections']):
        index += local_data['collections'][i]['floor_price'] * local_data['collections'][i]['weighting']
        delay += int(datetime.now().timestamp())-local_data['collections'][i]['last_update_timestamp']
    db_data['avg_delay'] = delay /(i+1)
    db_data['index'] =index
    return db_data
    
def appendDB(auth,db_data):
    headers = {
        'accept': 'application/json',
        'Authorization': f"{auth['token_type']} {auth['access_token']}",
        # Already added when you pass json= but not when you pass data=
        # 'Content-Type': 'application/json',
    }

    json_data = {
        'index': db_data['index'],
        'timestamp': db_data['current_timestamp'],
        'blocktime': db_data['block_number'],
        'avg_delay': db_data['avg_delay']
    }
    response = requests.post('http://127.0.0.1:8000/history/add', headers=headers, json=json_data)
    
local_data, auth = initLocalData()

while True:
    auth = login()
    updateComponents(local_data)
    db_data = feedDictConstructor(local_data)
    appendDB(auth,db_data)

Login Success...
getIndexList Success...
Login Success...
Updated otherdeed with FP 1.78 at 1665563098...
Updated proof-moonbirds with FP 10.27 at 1665563099...
Updated doodles-official with FP 8.1499 at 1665563100...
Updated goblintownwtf with FP 1.049 at 1665563101...
Updated azuki with FP 10.29 at 1665563102...
Updated pieceofshit-wtf with FP 0.07429 at 1665563102...
Updated cool-cats-nft with FP 3.633 at 1665563103...
Updated clonex with FP 8.87 at 1665563104...
Updated phantabear with FP 0.29 at 1665563104...
Updated beanzofficial with FP 0.94 at 1665563105...
Updated world-of-women-nft with FP 2.55 at 1665563106...
Updated mutant-ape-yacht-club with FP 14.0 at 1665563106...
Updated boredapeyachtclub with FP 74.2 at 1665563107...
Login Success...
Updated otherdeed with FP 1.78 at 1665563108...
Updated proof-moonbirds with FP 10.27 at 1665563109...
Updated doodles-official with FP 8.1499 at 1665563110...
Updated goblintownwtf with FP 1.049 at 1665563110...
Updated azuki with FP 10.

KeyboardInterrupt: 

Login Success...
getIndexList Success...
Login Success...
Updated otherdeed with FP 1.78 at 1665562890...
Updated proof-moonbirds with FP 10.26 at 1665562890...
Updated doodles-official with FP 8.1499 at 1665562891...
Updated goblintownwtf with FP 1.049 at 1665562892...
Updated azuki with FP 10.29 at 1665562893...
Updated pieceofshit-wtf with FP 0.07429 at 1665562893...
Updated cool-cats-nft with FP 3.6 at 1665562894...
Updated clonex with FP 8.87 at 1665562895...
Updated phantabear with FP 0.2899 at 1665562895...
Updated beanzofficial with FP 0.94 at 1665562896...
Updated world-of-women-nft with FP 2.589 at 1665562897...
Updated mutant-ape-yacht-club with FP 14.0 at 1665562897...
Updated boredapeyachtclub with FP 74.2 at 1665562898...
Login Success...
Updated otherdeed with FP 1.78 at 1665562899...
Updated proof-moonbirds with FP 10.27 at 1665562900...
Updated doodles-official with FP 8.15 at 1665562900...
Updated goblintownwtf with FP 1.049 at 1665562901...
Updated azuki with FP 10.2

KeyboardInterrupt: 

In [344]:
response.json()

{'index': 135.23729,
 'timestamp': 1665561008,
 'blocktime': 15730647,
 'avg_delay': 4.769230769230769}